# IPython NotebookファイルでPythonのSQLAlchemyを使用してPostgreSQLのデータを操作

## はじめに
- データベースはPostgreSQL13です
- 初めに以下のセルを実行してください
- セルに %%sql と記載することでSQLを発行することができます
- jupyterからはdescribeコマンドによるテーブル構造の確認ができないため、テーブル構造を確認する場合はlimitを指定したSELECTなどで代用してください
- 使い慣れたSQLクライアントを使っても問題ありません（接続情報は以下の通り）
  - IPアドレス：Docker Desktopの場合はlocalhost、Docker toolboxの場合は192.168.99.100
  - Port:5432
  - database名:analysis_postgres_db
  - ユーザ名：brain
  - パスワード:brain12345
- 大量出力を行うとJupyterが固まることがあるため、出力件数は制限することを推奨します
    - 結果確認のために表示させる量を適切にコントロールし、作業を軽快にすすめる技術もデータ加工には求められます
- 大量結果が出力された場合は、ファイルが重くなり以降開けなくなることもあります
    - vimエディタなどで大量出力範囲を削除することもできます
- 名前、住所等はダミーデータであり、実在するものではありません

In [2]:
# pythonからpostgresqlに接続するための準備
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

There's a new jupysql version available (0.10.10), you're running 0.7.2. To upgrade: pip install jupysql --upgrade


## 使い方
- セルの先頭に%%sqlと記載し、２行目以降にSQLを記述することでJupyterからPostgreSQLに対しSQLを実行できます。

In [7]:
%%sql
SELECT 'このように実行できます' AS sample;

*  postgresql://brain:***@db:5432/analysis_postgres_db
1 rows affected.


sample
このように実行できます


## 実践

In [8]:
%%sql
-- プランごとの新規ユーザー数を取得
SELECT 
  to_char(us.start_date, 'YYYY-MM') AS month,
  COUNT(CASE WHEN s.plan_name = 'basic' THEN 1 END) AS basic,
  COUNT(CASE WHEN s.plan_name = 'premium' THEN 1 END) AS premium
FROM
  user_subscriptions AS us
JOIN
  subscriptions AS s
ON 
  us.subscription_id = s.id
GROUP BY
  month
ORDER BY 
  month;

*  postgresql://brain:***@db:5432/analysis_postgres_db
7 rows affected.


month,basic,premium
2022-09,10,10
2022-10,10,12
2022-11,10,12
2022-12,10,8
2023-01,9,11
2023-02,9,11
2023-03,10,10
